In [ ]:
import torch
import torch.nn as nn # модуль, где определены слои для нейронных сетей
import torch.nn.utils.prune as prune
from torchvision import datasets, transforms, io, models
import torch.utils.data as data #для split-а
import torch.nn.functional as F # модуль, где определены активайии для слоев нейронных сетей

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

from google.colab import files

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt 
%matplotlib inline

from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import os
import gc
from glob import glob

from PIL import Image

# загрузка данных

https://www.kaggle.com/datasets/kmader/skin-cancer-mnist-ham10000

### предподготовка к загрузке датасетов

In [ ]:
# загрузка kaggle API в файле "kaggle.json"
files.upload()

# проверка актуальности загруженного API
!ls -lha kaggle.json

# The Kaggle API client expects this file to be in ~/.kaggle, so move it there.
!mkdir -p ~/.kaggle && mv kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
-rw-r--r-- 1 root root 65 May 24 16:35 kaggle.json


In [ ]:
# поиск необходимого датасета
!kaggle datasets list -s skin-cancer-mnist

# загрузка датасета
!kaggle datasets download -d kmader/skin-cancer-mnist-ham10000 

# разархивирование
!unzip skin-cancer-mnist-ham10000.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: ham10000_images_part_2/ISIC_0029325.jpg  
  inflating: ham10000_images_part_2/ISIC_0029326.jpg  
  inflating: ham10000_images_part_2/ISIC_0029327.jpg  
  inflating: ham10000_images_part_2/ISIC_0029328.jpg  
  inflating: ham10000_images_part_2/ISIC_0029329.jpg  
  inflating: ham10000_images_part_2/ISIC_0029330.jpg  
  inflating: ham10000_images_part_2/ISIC_0029331.jpg  
  inflating: ham10000_images_part_2/ISIC_0029332.jpg  
  inflating: ham10000_images_part_2/ISIC_0029333.jpg  
  inflating: ham10000_images_part_2/ISIC_0029334.jpg  
  inflating: ham10000_images_part_2/ISIC_0029335.jpg  
  inflating: ham10000_images_part_2/ISIC_0029336.jpg  
  inflating: ham10000_images_part_2/ISIC_0029337.jpg  
  inflating: ham10000_images_part_2/ISIC_0029338.jpg  
  inflating: ham10000_images_part_2/ISIC_0029339.jpg  
  inflating: ham10000_images_part_2/ISIC_0029340.jpg  
  inflating: ham10000_images_part_2/ISIC_0029341.jpg

In [ ]:
# список файлов в директории
!ls

ham10000_images_part_1	HAM10000_metadata.csv  hmnist_8_8_RGB.csv
HAM10000_images_part_1	hmnist_28_28_L.csv     sample_data
ham10000_images_part_2	hmnist_28_28_RGB.csv   skin-cancer-mnist-ham10000.zip
HAM10000_images_part_2	hmnist_8_8_L.csv


### чтение файла

In [ ]:
# папка с изображениями
data_dir = ''
all_image_path = glob(os.path.join(data_dir, '*', '*.jpg'))
imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x for x in all_image_path}

In [ ]:
# проверка вариативности размерности изобраджений
img_size_list = []
for i in tqdm(imageid_path_dict.values()):
  tsr_img = io.read_image(i)
  img_size_list.append(tsr_img.shape)

set(img_size_list)

  0%|          | 0/10015 [00:00<?, ?it/s]

{torch.Size([3, 450, 600])}

In [ ]:
# словарь 7-ми типов болезней
lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'dermatofibroma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

df_original = pd.read_csv('HAM10000_metadata.csv')

# добавляю поле полного названия
df_original['cell_type'] = df_original['dx'].map(lesion_type_dict.get)
# кодирование в цифровой формат
df_original['cell_type_idx'] = pd.Categorical(df_original['cell_type']).codes
# добавляю поле - путь к каждому изображению
df_original['path'] = df_original['image_id'].map(imageid_path_dict.get)

df_original.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,cell_type,cell_type_idx,path
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,Benign keratosis-like lesions,2,ham10000_images_part_1/ISIC_0027419.jpg
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,Benign keratosis-like lesions,2,ham10000_images_part_1/ISIC_0025030.jpg
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,Benign keratosis-like lesions,2,ham10000_images_part_1/ISIC_0026769.jpg
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,Benign keratosis-like lesions,2,ham10000_images_part_1/ISIC_0025661.jpg
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,Benign keratosis-like lesions,2,ham10000_images_part_2/ISIC_0031633.jpg


# преобразование данных

### проверка наличия дублей

In [ ]:
# новый датасет без дублей
df_undup = df_original.groupby('lesion_id').count()
# уникальность наблюдение-картинка
df_undup = df_undup[df_undup['image_id'] == 1]
df_undup.reset_index(inplace=True)

print('размерность до дедубликации:', df_original.shape)
print('размерность после дедубликации:', df_undup.shape)

размерность до дедубликации: (10015, 10)
размерность после дедубликации: (5514, 10)


In [ ]:
# создаем признак наличия дублей
def get_duplicates(x):
    unique_list = list(df_undup['lesion_id'])
    if x in unique_list:
        return 'unduplicated'
    else:
        return 'duplicated'
df_original['duplicates'] = df_original['lesion_id'].apply(get_duplicates)

# кол-во дублей
df_original['duplicates'].value_counts()

unduplicated    5514
duplicated      4501
Name: duplicates, dtype: int64

In [ ]:
# датасет без дублей
df_undup = df_original[df_original['duplicates'] == 'unduplicated']
df_undup.shape

(5514, 11)

### создание валидационного датасета

In [ ]:
y = df_undup['cell_type_idx'] # target-поле
_, df_val = train_test_split(df_undup, test_size=0.2, random_state=123, stratify=y)
df_val.shape

(1103, 11)

### создание обучающего датасета
##### - дубли допустимы
##### - не пересекается с валидационным датасетом

In [ ]:
# проверка на наличие картинки в валидационном датасете
def get_val_rows(x):
    val_list = list(df_val['image_id'])
    if str(x) in val_list:
        return 'val'
    else:
        return 'train'

df_original['train_or_val'] = df_original['image_id'].apply(get_val_rows)
df_train = df_original[df_original['train_or_val'] == 'train']
print(len(df_train))

8912


### распределение в обучающем датасете

In [ ]:
target_distrib = df_train['cell_type_idx'].value_counts().sort_index()
target_distrib

0     297
1     479
2    1011
3     107
4    5822
5     129
6    1067
Name: cell_type_idx, dtype: int64

### обучающий датасет несбалансирован, поэтому увеличим его, чтобы выровнять и получить равномерное распределение

In [ ]:
# настройка величины увеличения наблюдений в соответствии с их соотношением в датасете
max_blnc = max(target_distrib.values)
data_aug_rate = [ int((max_blnc - i)/i) if max_blnc != i else 0 for i in target_distrib.values ]
print(data_aug_rate)

# oversampling
for i in range(len(target_distrib)):
    if data_aug_rate[i]:
        df_train=df_train.append([df_train.loc[df_train['cell_type_idx'] == i,:]]*(data_aug_rate[i]-1), ignore_index=True)
df_train['cell_type_idx'].value_counts()

[18, 11, 4, 53, 0, 44, 4]


4    5822
5    5676
3    5671
0    5346
1    5269
6    4268
2    4044
Name: cell_type_idx, dtype: int64

### теперь мы имеем обучающий датасет с равномерно распределенными событиями

In [ ]:
# обновим индексы
df_train = df_train.reset_index()
df_val = df_val.reset_index()

In [ ]:
# # norm_mean = (0.49139968, 0.48215827, 0.44653124)
# # norm_std = (0.24703233, 0.24348505, 0.26158768)
# # define the transformation of the train images.
# train_transform = transforms.Compose([transforms.Resize((input_size,input_size)),
#                                       transforms.RandomHorizontalFlip(),
#                                       transforms.RandomVerticalFlip(),
#                                       transforms.RandomRotation(20),
#                                       transforms.ColorJitter(brightness=0.1, contrast=0.1, hue=0.1),
#                                         transforms.ToTensor(), 
#                                       transforms.Normalize(norm_mean, norm_std)])
# # define the transformation of the val images.
# val_transform = transforms.Compose([transforms.Resize((input_size,input_size)), 
#                                     transforms.ToTensor(),
#                                     transforms.Normalize(norm_mean, norm_std)])

In [ ]:
# преобразования изображений для лучшего обучения
train_transforms = transforms.Compose([transforms.Resize(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.RandomVerticalFlip(),
                                       transforms.RandomRotation(30),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

val_transforms = transforms.Compose([transforms.Resize(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [ ]:
# класс объединяющий изображение с меткой
class createDataset(data.Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        # загрузка изображения и получения его метки
        X = Image.open(self.df['path'][index])
        y = torch.tensor(int(self.df['cell_type_idx'][index]))

        if self.transform:
            X = self.transform(X)

        return X, y

In [ ]:
# создание train-датасета
train_dataset = createDataset(df_train, transform=train_transforms)

# создание генератора - DataLoader для train-датасета
train_loader = data.DataLoader(train_dataset, batch_size=32, shuffle=True)

# создание validate-датасета
val_dataset = createDataset(df_val , transform = val_transforms)

# создание генератора - DataLoader для validate-датасета
val_loader = data.DataLoader(val_dataset,batch_size=32, shuffle=False)

In [ ]:
train_dataset.__getitem__(0)

(tensor([[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          ...,
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],
 
         [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          ...,
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],
 
         [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
          [-1.8044, -1.8044,

# нейросеть

In [ ]:
device = torch.device('cuda')
print( torch.cuda.is_available() )
print( torch.cuda.get_device_name() )

True
Tesla T4


In [ ]:
output_size = 7 # кол-во классов

### функция обучения сети

In [ ]:
def train(net, n_epoch=1):
    # выбираем функцию потерь
    loss_fn = torch.nn.CrossEntropyLoss()

    # выбираем алгоритм оптимизации и learning_rate
    learning_rate = 1e-3
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

    # acc по test
    best_accuracy = 0
    # обучаем сеть 2 эпохи
    for epoch in tqdm(range(n_epoch)):

        running_loss = 0.0
        train_dataiter = iter(train_loader)
        for i, batch in enumerate(tqdm(train_dataiter)):
            # так получаем текущий батч
            X_batch, y_batch = batch
            
            # переносим его на видеопамять
            # если точно уверены, что это гпу - можно написать .cuda()
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            # обнуляем градиент
            optimizer.zero_grad()

            # forward pass (получение ответов на батч картинок)
            y_pred = net(X_batch)
            # вычисление лосса от выданных сетью ответов и правильных ответов на батч
            loss = loss_fn(y_pred, y_batch)
            # bаckpropagation (вычисление градиентов)
            loss.backward()
            # обновление весов сети
            optimizer.step()
            
            # проверяем качество каждые 100 батчей
            if i % 200 ==0:
                # менеджер упарвления контекстом торча указывает на то, чтобы не обновлять параметры
                with torch.no_grad():
                    accuracy = []
                    for batch in val_loader:
                        x, y = batch
                        # переносим на гпу
                        x = x.to(device)
                        y = y.to(device)
                        # прогнозируем
                        y_pred = net(x)
                        # loss = loss_fn(y_pred, y)
                        # находим accuracy батча с теста
                        accuracy.append(accuracy_score(y.detach().cpu().numpy(), np.argmax(y_pred.detach().cpu().numpy(), axis=1)))
                    # усредняем accuracy всех батчей на тесте
                    accuracy = np.mean(np.array(accuracy))
                    # если стало лучше - сохраняем на диск и обновляем лучшую метрику
                    if accuracy > best_accuracy:
                        print('New best model with test acc:', accuracy)
                        torch.save(net.state_dict(), './best_model.pt')
                        best_accuracy = accuracy

            # выведем текущий loss
            running_loss += loss.item()
            # выведем качество каждые 500 батчей
            if i % 100 == 99:
                print('[%d, %5d] loss: %.3f' %
                        (epoch + 1, i + 1, running_loss / 500))
                running_loss = 0.0

    print('Обучение закончено')
    return net

### 1) vgg16

In [ ]:
vgg16 = models.vgg16(pretrained=True)
vgg16.parameters

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

<bound method Module.parameters of VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size

In [ ]:
# нам не нужен последний слой, обрезаем его
vgg16.classifier = nn.Sequential(*list(vgg16.classifier.children()))[:-1]
vgg16.parameters

<bound method Module.parameters of VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size

In [ ]:
class New_VGG16(nn.Module):
    def __init__(self):
        super().__init__()
        self.vgg16 = vgg16 # вся огромная нейросеть в одну строчку
        for param in self.vgg16.features.parameters(): # отключаем для нее обновление параметров
            param.requires_grad = False
        self.fc = nn.Linear(4096, output_size) # добавляем новый слой
    
    def forward(self, x):
        # forward pass сети
        # умножение на матрицу весов 1 слоя и применение функции активации
        x = self.vgg16(x)
        # print(x.size())
        x = self.fc(x)
        return x

In [ ]:
net = New_VGG16().to(device)
train(net)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1128 [00:00<?, ?it/s]

New best model with test acc: 0.05011904761904762
[1,   100] loss: 0.387
[1,   200] loss: 0.339
New best model with test acc: 0.6028571428571429
[1,   300] loss: 0.306
[1,   400] loss: 0.300
[1,   500] loss: 0.290
[1,   600] loss: 0.294
[1,   700] loss: 0.274
[1,   800] loss: 0.274
[1,   900] loss: 0.263
[1,  1000] loss: 0.265
[1,  1100] loss: 0.253
Обучение закончено


New_VGG16(
  (vgg16): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (16)

In [ ]:
with torch.no_grad():
    accuracy = []
    for batch in val_loader:
        x, y = batch
        # переносим на гпу
        x = x.to(device)
        y = y.to(device)
        # прогнозируем
        y_pred = net(x)
        # loss = loss_fn(y_pred, y)
        # находим accuracy батча с теста
        accuracy.append(accuracy_score(y.detach().cpu().numpy(), np.argmax(y_pred.detach().cpu().numpy(), axis=1)))
    # усредняем accuracy всех батчей на тесте
    accuracy = np.mean(np.array(accuracy))

print('Test accuracy', accuracy)

Test accuracy 0.6296428571428572


### 2) vgg16 - много эпох

In [ ]:
net_long = New_VGG16().to(device)
train(net_long, n_epoch=100)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/1128 [00:00<?, ?it/s]

New best model with test acc: 0.11529761904761904
[1,   100] loss: 0.245
[1,   200] loss: 0.234
New best model with test acc: 0.5385714285714286
[1,   300] loss: 0.241
[1,   400] loss: 0.240
New best model with test acc: 0.6200595238095238
[1,   500] loss: 0.258
[1,   600] loss: 0.254
New best model with test acc: 0.7050000000000001
[1,   700] loss: 0.236
[1,   800] loss: 0.232
[1,   900] loss: 0.236
[1,  1000] loss: 0.219
[1,  1100] loss: 0.222


  0%|          | 0/1128 [00:00<?, ?it/s]

[2,   100] loss: 0.223
[2,   200] loss: 0.224
[2,   300] loss: 0.215
[2,   400] loss: 0.229
[2,   500] loss: 0.222
[2,   600] loss: 0.215
[2,   700] loss: 0.218
[2,   800] loss: 0.215
[2,   900] loss: 0.213
[2,  1000] loss: 0.207
[2,  1100] loss: 0.217


...

  0%|          | 0/1128 [00:00<?, ?it/s]

[10,   100] loss: 0.168
[10,   200] loss: 0.181
[10,   300] loss: 0.175
[10,   400] loss: 0.175
[10,   500] loss: 0.168
[10,   600] loss: 0.172
[10,   700] loss: 0.167
[10,   800] loss: 0.172
[10,   900] loss: 0.169
[10,  1000] loss: 0.189
[10,  1100] loss: 0.176


  0%|          | 0/1128 [00:00<?, ?it/s]

[11,   100] loss: 0.159
[11,   200] loss: 0.167
[11,   300] loss: 0.161
[11,   400] loss: 0.163
[11,   500] loss: 0.165
[11,   600] loss: 0.170
[11,   700] loss: 0.164
[11,   800] loss: 0.168
[11,   900] loss: 0.164
[11,  1000] loss: 0.179
[11,  1100] loss: 0.166


  0%|          | 0/1128 [00:00<?, ?it/s]

In [ ]:
with torch.no_grad():
    accuracy = []
    for batch in val_loader:
        x, y = batch
        # переносим на гпу
        x = x.to(device)
        y = y.to(device)
        # прогнозируем
        y_pred = net_long(x)
        # loss = loss_fn(y_pred, y)
        # находим accuracy батча с теста
        accuracy.append(accuracy_score(y.detach().cpu().numpy(), np.argmax(y_pred.detach().cpu().numpy(), axis=1)))
    # усредняем accuracy всех батчей на тесте
    accuracy = np.mean(np.array(accuracy))

print('Test accuracy', accuracy)

### 3) resnet50

In [ ]:
resnet50 = models.resnet50(pretrained=True)

resnet50.parameters
resnet50.fc
num_ftrs = resnet50.fc.in_features

resnet50.fc = torch.nn.Linear(num_ftrs, output_size)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [ ]:
class New_resnet50(nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet50 = resnet50 # вся огромная нейросеть в одну строчку
    
    def forward(self, x):
        # forward pass сети
        # умножение на матрицу весов 1 слоя и применение функции активации
        x = self.resnet50(x)
        return x

In [ ]:
net_2 = New_resnet50().to(device)
train(net_2)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1128 [00:00<?, ?it/s]

New best model with test acc: 0.3536309523809524
[1,   100] loss: 0.271
[1,   200] loss: 0.220
New best model with test acc: 0.4828571428571428
[1,   300] loss: 0.205
[1,   400] loss: 0.187
[1,   500] loss: 0.171
[1,   600] loss: 0.163
[1,   700] loss: 0.157
[1,   800] loss: 0.144
[1,   900] loss: 0.148
[1,  1000] loss: 0.137
[1,  1100] loss: 0.132
Обучение закончено


New_resnet50(
  (resnet50): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
    

In [ ]:
with torch.no_grad():
    accuracy = []
    for batch in val_loader:
        x, y = batch
        # переносим на гпу
        x = x.to(device)
        y = y.to(device)
        # прогнозируем
        y_pred = net_2(x)
        # loss = loss_fn(y_pred, y)
        # находим accuracy батча с теста
        accuracy.append(accuracy_score(y.detach().cpu().numpy(), np.argmax(y_pred.detach().cpu().numpy(), axis=1)))
    # усредняем accuracy всех батчей на тесте
    accuracy = np.mean(np.array(accuracy))

print('Test accuracy', accuracy)

Test accuracy 0.45583333333333337


# прунинг

In [ ]:
import torch.nn.utils.prune as prune

net_pr = models.vgg16(pretrained=True)
for name, module in net_pr.named_modules():
    print(module)
    # # prune 20% of connections in all 2D-conv layers
    # if isinstance(module, torch.nn.Conv2d):
    #     prune.l1_unstructured(module, name='weight', amount=0.2)
    prune.random_unstructured(module, name="weight", amount=0.2)

print(dict(net_pr.named_buffers()).keys()) 

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

AttributeError: ignored

In [ ]:
train(net_pr)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1128 [00:00<?, ?it/s]

RuntimeError: ignored

In [ ]:
with torch.no_grad():
    accuracy = []
    for batch in val_loader:
        x, y = batch
        # переносим на гпу
        x = x.to(device)
        y = y.to(device)
        # прогнозируем
        y_pred = net_pr(x)
        # loss = loss_fn(y_pred, y)
        # находим accuracy батча с теста
        accuracy.append(accuracy_score(y.detach().cpu().numpy(), np.argmax(y_pred.detach().cpu().numpy(), axis=1)))
    # усредняем accuracy всех батчей на тесте
    accuracy = np.mean(np.array(accuracy))

print('Test accuracy', accuracy)

RuntimeError: ignored